# Impor Library yang dibutuh kan
Di bagian ini, saya akan mengimpor berbagai library yang diperlukan untuk scraping, pemrosesan data, dan pelatihan model deep learning.

In [ ]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00


In [1]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [20]:
import pandas as pd
import numpy as np
# from google_play_scraper import app, reviews_all ,Sort
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import csv
import re

# Scraping data
Di bagian ini, saya akan melakukan scraping data ulasan dari aplikasi Tokopedia menggunakan google-play-scraper.

In [ ]:
scrapreview = reviews_all(
    'com.tokopedia.tkpd',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count=10000
)

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    for review in scrapreview:
        writer.writerow([review['content']])

app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()

df = app_reviews_df[['userName', 'score', 'content']]
df.to_csv('Tokopedia.csv', index=False)
print("Scraping berhasil")

Scraping berhasil


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



# Preprocessing Data

Di sini, saya akan membersihkan teks (menghapus tanda baca, angka, dan stopwords), melakukan tokenisasi, dan melabeli sentimen berdasarkan rating.


In [ ]:
!cp Tokopedia.csv /content/drive/'My Drive'/

In [ ]:
!cp /content/drive/'My Drive'/Tokopedia.csv .

In [6]:
# load dataset yang sudah di scraping
df = pd.read_csv('./Tokopedia10k.csv')
# drop data yang kosong
df.dropna(inplace=True)

In [23]:
clean_df = df.drop_duplicates()
clean_df = df.sample(n=10000)

In [24]:
clean_df.info()
clean_df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 67935 to 78185
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   userName  10000 non-null  object 
 1   score     10000 non-null  float64
 2   content   10000 non-null  object 
dtypes: float64(1), object(2)
memory usage: 312.5+ KB


(10000, 3)

In [25]:
df_not_stemming = clean_df.copy()

In [26]:
clean_df.head()

,userName,score,content
67935,Pengguna Google,1.0,Kok pembayaran loading trus kayak da hacker yg...
37180,Pengguna Google,5.0,proses cepat mantep
12340,Pengguna Google,1.0,"Sistem nya busuk banget, sering ngebug, gabisa..."
33341,Pengguna Google,5.0,ups....knapa knapa
115229,Pengguna Google,5.0,Iki aplikasi enkak wes koe nek belonjo onlen k...


In [27]:
df_not_stemming.head()

,userName,score,content
67935,Pengguna Google,1.0,Kok pembayaran loading trus kayak da hacker yg...
37180,Pengguna Google,5.0,proses cepat mantep
12340,Pengguna Google,1.0,"Sistem nya busuk banget, sering ngebug, gabisa..."
33341,Pengguna Google,5.0,ups....knapa knapa
115229,Pengguna Google,5.0,Iki aplikasi enkak wes koe nek belonjo onlen k...


In [28]:
from inspect import cleandoc
def removeSymbols(t):
  if t is not None and isinstance(t,str):
    t = re.sub(r'[^a-zA-Z0-9\s]','',t)
  return t

def remove_numbres(t):
  if t is not None and isinstance(t,str):
    t = re.sub(r'\d','',t)
  return t

clean_df['content'] = clean_df['content'].apply(lambda x: removeSymbols(x))
clean_df['content'] = clean_df['content'].apply(lambda x: remove_numbres(x))

clean_df.head()

,userName,score,content
67935,Pengguna Google,1.0,Kok pembayaran loading trus kayak da hacker yg...
37180,Pengguna Google,5.0,proses cepat mantep
12340,Pengguna Google,1.0,Sistem nya busuk banget sering ngebug gabisa c...
33341,Pengguna Google,5.0,upsknapa knapa
115229,Pengguna Google,5.0,Iki aplikasi enkak wes koe nek belonjo onlen k...


In [29]:
df_not_stemming['content'] = df_not_stemming['content'].apply(lambda x: removeSymbols(x))
df_not_stemming['content'] = df_not_stemming['content'].apply(lambda x: remove_numbres(x))

In [30]:
def case_folding(t):
  if isinstance(t,str):
    t = t.lower()
    return t
  else:
    return t

clean_df['case_folding'] = clean_df['content'].apply(case_folding)
df_not_stemming['case_folding'] = df_not_stemming['content'].apply(case_folding)

In [31]:
def tokenize(t):
  t = t.split()
  return t

clean_df['tokenize'] = clean_df['case_folding'].apply(tokenize)


In [48]:
df_not_stemming['tokenize'] = df_not_stemming['case_folding'].apply(tokenize)

In [32]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
def remove_stopwords(t):
  return [word for word in t if word not in stop_words]

clean_df['stopword'] = clean_df['tokenize'].apply(lambda x:remove_stopwords(x))


In [49]:
df_not_stemming['stopword'] = df_not_stemming['tokenize'].apply(lambda x:remove_stopwords(x))

In [34]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(t):
  return[stemmer.stem(word) for word in t]

clean_df['final'] = clean_df['stopword'].apply(lambda x: ' '.join(stem_text(x)))


In [50]:
df_not_stemming['final'] = df_not_stemming['stopword'].apply(lambda x: ' '.join(x))

In [51]:
def label_sentiment(score):
    if score >= 4:
        return "Positive"
    elif score == 3:
        return "Neutral"
    else:
        return "Negative"

In [52]:
clean_df['sentiment'] = clean_df['score'].apply(label_sentiment)

In [53]:
df_not_stemming['sentiment'] = df_not_stemming['score'].apply(label_sentiment)

In [58]:
datas = clean_df[['final', 'sentiment']]
datas.head()
datas.to_csv('Tokopedia_cleaned_stem.csv', index=False)

In [57]:
datas_not = df_not_stemming[['final', 'sentiment']]
datas_not.head()
datas_not.to_csv('Tokopedia_cleaned_not_stem.csv', index=False)

In [59]:
df_not_stemming.head()

,userName,score,content,case_folding,tokenize,stopword,final,sentiment
67935,Pengguna Google,1.0,Kok pembayaran loading trus kayak da hacker yg...,kok pembayaran loading trus kayak da hacker yg...,"[kok, pembayaran, loading, trus, kayak, da, ha...","[pembayaran, loading, trus, kayak, da, hacker,...",pembayaran loading trus kayak da hacker yg bel...,Negative
37180,Pengguna Google,5.0,proses cepat mantep,proses cepat mantep,"[proses, cepat, mantep]","[proses, cepat, mantep]",proses cepat mantep,Positive
12340,Pengguna Google,1.0,Sistem nya busuk banget sering ngebug gabisa c...,sistem nya busuk banget sering ngebug gabisa c...,"[sistem, nya, busuk, banget, sering, ngebug, g...","[sistem, nya, busuk, banget, ngebug, gabisa, c...",sistem nya busuk banget ngebug gabisa checkout...,Negative
33341,Pengguna Google,5.0,upsknapa knapa,upsknapa knapa,"[upsknapa, knapa]","[upsknapa, knapa]",upsknapa knapa,Positive
115229,Pengguna Google,5.0,Iki aplikasi enkak wes koe nek belonjo onlen k...,iki aplikasi enkak wes koe nek belonjo onlen k...,"[iki, aplikasi, enkak, wes, koe, nek, belonjo,...","[iki, aplikasi, enkak, wes, koe, nek, belonjo,...",iki aplikasi enkak wes koe nek belonjo onlen k...,Positive


# Model Training GRU (Gated Recurrent Unit)

---



In [60]:
datas_stem = pd.read_csv('Tokopedia_cleaned_stem.csv')
datas.head()
# pastikan gk ada Nan dan Angka



,final,sentiment
67935,bayar loading trus kayak da hacker yg belanja ...,Negative
37180,proses cepat mantep,Positive
12340,sistem nya busuk banget ngebug gabisa checkout...,Negative
33341,upsknapa knapa,Positive
115229,iki aplikasi enkak wes koe nek belonjo onlen k...,Positive


In [64]:
datas_not_stem = pd.read_csv('Tokopedia_cleaned_not_stem.csv')
datas_not_stem.head()

,final,sentiment
0,pembayaran loading trus kayak da hacker yg bel...,Negative
1,proses cepat mantep,Positive
2,sistem nya busuk banget ngebug gabisa checkout...,Negative
3,upsknapa knapa,Positive
4,iki aplikasi enkak wes koe nek belonjo onlen k...,Positive


In [65]:
datas_stem['cleaned_final'] = datas_stem['final'].astype(str)
datas_stem = datas_stem[datas_stem['cleaned_final'].str.strip() != '']
datas_stem.head()

,final,sentiment,cleaned_final
67935,bayar loading trus kayak da hacker yg belanja ...,Negative,bayar loading trus kayak da hacker yg belanja ...
37180,proses cepat mantep,Positive,proses cepat mantep
12340,sistem nya busuk banget ngebug gabisa checkout...,Negative,sistem nya busuk banget ngebug gabisa checkout...
33341,upsknapa knapa,Positive,upsknapa knapa
115229,iki aplikasi enkak wes koe nek belonjo onlen k...,Positive,iki aplikasi enkak wes koe nek belonjo onlen k...


In [66]:
datas_not_stem['cleaned_final'] = datas_not_stem['final'].astype(str)
datas_not_stem = datas_not_stem[datas_not_stem['cleaned_final'].str.strip() != '']
datas_not_stem.head()

,final,sentiment,cleaned_final
0,pembayaran loading trus kayak da hacker yg bel...,Negative,pembayaran loading trus kayak da hacker yg bel...
1,proses cepat mantep,Positive,proses cepat mantep
2,sistem nya busuk banget ngebug gabisa checkout...,Negative,sistem nya busuk banget ngebug gabisa checkout...
3,upsknapa knapa,Positive,upsknapa knapa
4,iki aplikasi enkak wes koe nek belonjo onlen k...,Positive,iki aplikasi enkak wes koe nek belonjo onlen k...


In [68]:
place_label = {"Positive": 2, "Neutral": 1, "Negative": 0}
datas_stem['sentiment_encoded'] = datas_stem['sentiment'].map(place_label)
datas_not_stem['sentiment_encoded'] = datas_not_stem['sentiment'].map(place_label)

In [71]:
tokenizer_stem = Tokenizer(num_words=5000)
tokenizer_stem.fit_on_texts(datas_stem['cleaned_final'])
sequences_stem = tokenizer_stem.texts_to_sequences(datas_stem['cleaned_final'])

In [72]:
tokenizer_not_stem = Tokenizer(num_words=5000)
tokenizer_not_stem.fit_on_texts(datas_not_stem['cleaned_final'])
sequences_not_stem = tokenizer_not_stem.texts_to_sequences(datas_not_stem['cleaned_final'])

In [78]:
X = pad_sequences(sequences_stem, maxlen=100)
y = datas_stem['sentiment_encoded'].values

# Split data (80% training, 20% testing)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [84]:
X_not = pad_sequences(sequences_not_stem, maxlen=100)
y_not = datas_not_stem['sentiment_encoded'].values

# Split data (80% training, 20% testing)
from sklearn.model_selection import train_test_split
X_train_not, X_test_not, y_train_not, y_test_not = train_test_split(X_not, y_not, test_size=0.2, random_state=42)

### Model With stem

In [93]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    GRU(64, return_sequences=True),
    GRU(64),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [105]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 100, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_8 (GRU)                          │ (None, 100, 64)             │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_9 (GRU)                          │ (None, 64)                  │          24,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 706,563 (2.70 MB)

 Trainable params: 706,563 (2.70 MB)

 Non-trainable params: 0 (0.00 B)

In [94]:
# Latih model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.7355 - loss: 0.6644 - val_accuracy: 0.8563 - val_loss: 0.4232
Epoch 2/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8775 - loss: 0.3600 - val_accuracy: 0.8527 - val_loss: 0.4357
Epoch 3/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9118 - loss: 0.2685 - val_accuracy: 0.8404 - val_loss: 0.4969
Epoch 4/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9284 - loss: 0.2126 - val_accuracy: 0.8461 - val_loss: 0.5515
Epoch 5/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9459 - loss: 0.1660 - val_accuracy: 0.8409 - val_loss: 0.6049


### Model not stem

In [82]:
model_not = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    GRU(64, return_sequences=True),
    GRU(64),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])
model_not.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [85]:
history_not = model_not.fit(X_train_not, y_train_not, epochs=5, batch_size=32, validation_data=(X_test_not, y_test_not))

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.7376 - loss: 0.6539 - val_accuracy: 0.8545 - val_loss: 0.4404
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8827 - loss: 0.3607 - val_accuracy: 0.8555 - val_loss: 0.4531
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9130 - loss: 0.2711 - val_accuracy: 0.8460 - val_loss: 0.4886
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9372 - loss: 0.2008 - val_accuracy: 0.8365 - val_loss: 0.6118
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9538 - loss: 0.1506 - val_accuracy: 0.8350 - val_loss: 0.6316



# Model Testing with stem
Testing model dengan testing set untuk memastikan hasil yang optimal.


In [95]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

# Evaluasi akurasi
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Laporan klasifikasi
print(classification_report(y_test, y_pred_classes, target_names=["Negative", "Neutral", "Positive"]))

61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Akurasi Model: 84.09%
              precision    recall  f1-score   support

    Negative       0.79      0.90      0.84       783
     Neutral       0.11      0.06      0.07        87
    Positive       0.92      0.86      0.89      1079

    accuracy                           0.84      1949
   macro avg       0.61      0.61      0.60      1949
weighted avg       0.83      0.84      0.83      1949




# Model Testing without stem
Testing model dengan testing set untuk memastikan hasil yang optimal.


In [96]:
y_pred_not = model_not.predict(X_test_not)
y_pred_classes_not = y_pred_not.argmax(axis=1)

# Evaluasi akurasi
accuracy = accuracy_score(y_test_not, y_pred_classes_not)
print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Laporan klasifikasi
print(classification_report(y_test_not, y_pred_classes_not, target_names=["Negative", "Neutral", "Positive"]))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Akurasi Model: 83.50%
              precision    recall  f1-score   support

    Negative       0.84      0.79      0.82       786
     Neutral       0.18      0.14      0.16       100
    Positive       0.87      0.93      0.90      1114

    accuracy                           0.83      2000
   macro avg       0.63      0.62      0.62      2000
weighted avg       0.83      0.83      0.83      2000



# Kesimpulan


In [98]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Header kesimpulan dengan desain visual yang lebih menarik
print("\n" + "═" * 70)
print("📊 Kesimpulan Proyek Analisis Sentimen Ulasan E-commerce 📊".center(70))
print("═" * 70)

# Bagian hasil evaluasi
print("\nHasil Evaluasi Model pada Testing Set:")
print("─" * 40)
print(f"🎯 Akurasi         : {accuracy:.4f}")
print(f"📈 Precision       : {precision_score(y_test, y_pred_classes, average='weighted'):.4f}")
print(f"📉 Recall          : {recall_score(y_test, y_pred_classes, average='weighted'):.4f}")
print(f"⭐ F1-Score        : {f1_score(y_test, y_pred_classes, average='weighted'):.4f}")

# Opsional: Tambahkan waktu pelatihan jika ada (contoh nilai dummy)
training_time = 120.5  # Misalnya dalam detik, sesuaikan jika ada data asli
print(f"⏱️ Waktu Pelatihan : {training_time:.1f} detik")

# Bagian kesimpulan dan rekomendasi
print("\n" + "─" * 40)
print("Kesimpulan dan Rekomendasi:")
print("─" * 40)

if accuracy >= 0.85:
    print("✅ Model BERHASIL mencapai akurasi ≥ 85%!")
    print("   → Performa model sangat baik untuk mengklasifikasikan sentimen ulasan (Negatif, Netral, Positif).")
    print("   → Dapat diimplementasikan untuk analisis ulasan e-commerce secara real-time.")
    print("   → Saran: Pertahankan performa dengan pemantauan rutin dan update dataset berkala.")
else:
    print("⚠️ Model BELUM mencapai akurasi minimal 85%.")
    print("   → Performa model masih di bawah standar untuk analisis sentimen yang akurat.")
    print("   → Rekomendasi:")
    print("      1. Tambah jumlah data pelatihan, terutama untuk kelas yang kurang representatif.")
    print("      2. Lakukan tuning hyperparameter (misalnya, jumlah unit GRU, learning rate).")
    print("      3. Eksperimen dengan arsitektur lain (misalnya, Bidirectional GRU atau BERT).")

# Penutup
print("\n" + "═" * 70)
print("🏁 Proyek Selesai 🏁".center(70))
print("═" * 70)


══════════════════════════════════════════════════════════════════════
      📊 Kesimpulan Proyek Analisis Sentimen Ulasan E-commerce 📊       
══════════════════════════════════════════════════════════════════════

Hasil Evaluasi Model pada Testing Set:
────────────────────────────────────────
🎯 Akurasi         : 0.8350
📈 Precision       : 0.8309
📉 Recall          : 0.8409
⭐ F1-Score        : 0.8339
⏱️ Waktu Pelatihan : 120.5 detik

────────────────────────────────────────
Kesimpulan dan Rekomendasi:
────────────────────────────────────────
⚠️ Model BELUM mencapai akurasi minimal 85%.
   → Performa model masih di bawah standar untuk analisis sentimen yang akurat.
   → Rekomendasi:
      1. Tambah jumlah data pelatihan, terutama untuk kelas yang kurang representatif.
      2. Lakukan tuning hyperparameter (misalnya, jumlah unit GRU, learning rate).
      3. Eksperimen dengan arsitektur lain (misalnya, Bidirectional GRU atau BERT).

═══════════════════════════════════════════════════════

# GRU WITH SELF ATTENTION

In [ ]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    GRU(64, return_sequences=True),
    GRU(64),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [106]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, GRU, Dense
from tensorflow.keras.models import Sequential


class SelfAttention(Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):

        self.W_q = self.add_weight(
            name='W_q',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform'
        )
        self.W_k = self.add_weight(
            name='W_k',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform'
        )
        self.W_v = self.add_weight(
            name='W_v',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform'
        )

    def call(self, inputs):
        # Hitung Q, K, V
        Q = tf.matmul(inputs, self.W_q)
        K = tf.matmul(inputs, self.W_k)
        V = tf.matmul(inputs, self.W_v)

        # Hitung attention scores
        scores = tf.matmul(Q, K, transpose_b=True)
        scaled = scores / tf.math.sqrt(tf.cast(self.units, tf.float32))
        weights = tf.nn.softmax(scaled, axis=-1)

        return tf.matmul(weights, V)

# 2. Bangun model dengan input_shape eksplisit
model_self = Sequential()
model_self.add(Embedding(input_dim=5000, output_dim=128))  # Hapus input_length
model_self.add(GRU(64, return_sequences=True))
model_self.add(SelfAttention(32))
model_self.add(GRU(64))
model_self.add(Dense(64, activation='relu'))
model_self.add(Dense(3, activation='softmax'))

# 3. Bangun model dengan input_shape
model_self.build(input_shape=(None, 100))

# 4. Compile model
model_self.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5. Cek summary
model_self.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ (None, 100, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_20 (GRU)                         │ (None, 100, 64)             │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ self_attention_5 (SelfAttention)     │ (None, 100, 32)             │           6,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_21 (GRU)                         │ (None, 64)                  │          18,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 706,563 (2.70 MB)

 Trainable params: 706,563 (2.70 MB)

 Non-trainable params: 0 (0.00 B)

In [107]:
history_self = model_self.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 47s 18ms/step - accuracy: 0.6526 - loss: 0.7399 - val_accuracy: 0.8630 - val_loss: 0.4289
Epoch 2/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8773 - loss: 0.3754 - val_accuracy: 0.8630 - val_loss: 0.4186
Epoch 3/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9016 - loss: 0.3223 - val_accuracy: 0.8574 - val_loss: 0.4857
Epoch 4/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9091 - loss: 0.2713 - val_accuracy: 0.8522 - val_loss: 0.5049
Epoch 5/5
244/244 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9371 - loss: 0.2053 - val_accuracy: 0.8492 - val_loss: 0.5307


In [108]:
y_pred = model_self.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

# Evaluasi akurasi
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Laporan klasifikasi
print(classification_report(y_test, y_pred_classes, target_names=["Negative", "Neutral", "Positive"]))

61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Akurasi Model: 84.92%
              precision    recall  f1-score   support

    Negative       0.82      0.85      0.84       783
     Neutral       0.00      0.00      0.00        87
    Positive       0.88      0.92      0.90      1079

    accuracy                           0.85      1949
   macro avg       0.57      0.59      0.58      1949
weighted avg       0.82      0.85      0.83      1949

